In [1]:
import math
import numpy as np

### Morpheme  

Represents a single morpheme, the smallest unit of meaning in a word. Stores details like its type (prefix, suffix), frequency count, probability score, and whether it's the final part of a segmentation. Also includes methods to compare and hash morphemes for use in sets and dictionaries.  


In [2]:
class Morpheme():
    def __init__(self,morpheme:str, type, ending:bool, count:int = 1, probability:float = 0.0)->None:
        self.morpheme = morpheme
        self.ending = ending 
        self.count = count 
        self.type = type
        self.probability = probability

    def __str__(self) -> str:
        return f"morpheme: {self.morpheme}, Count: {self.count}, Probability: {self.probability:.4f}, ending: {self.ending}"
    def __hash__(self) ->int:
        return hash(self.morpheme)
    def __eq__(self, other) ->bool:
        return isinstance(other, Morpheme) and self.morpheme == other.morpheme
        

### GraphNode  

Represents a node in the transition graph. Each node holds a set of morphemes that can appear at a specific position in a word segmentation. Helps organize morphemes based on their position in the structure.  


In [3]:
class GraphNode():
    def __init__(self,index:int) ->None:
        self.index = index
        self.morphemes = set() 
        


### TransitionGraph  

Handles building and managing the transition graph for morphological segmentation. Reads the segmentation file, processes morphemes, and constructs a structure for generating possible inflected forms. Also normalizes morpheme probabilities and retrieves valid segmentations from the graph.  


In [4]:
class TransitionGraph():
    def __init__(self, inputFile:str)->None:
        self.graph =  []
        self.starting_node = [] 
        self.morpheme_index_mapping = {}
        self.morpheme_count = 0 
        self.build_transition_graph(inputFile)
        
    def build_transition_graph(self, inputFile:str)->None:
        if inputFile == None:
            raise ValueError("Please provide a valid input file path ")
        try:
            with open (inputFile, "r") as segmentations:
                for line in segmentations:
                    if line[0] == '+':
                        self.process_segmentation(line)
        except FileNotFoundError:
            raise FileNotFoundError(f"Error: The file '{inputFile}; was not found,")
        except IOError:
            raise IOError(f"Error: Colud not read the file '{inputFile}'")
        
        for node in self.graph:
            self.normalize(node)
            
    def process_segmentation(self, segmentation: str)->None:
        root_processed = False
        segments = segmentation[1:].split("+")
        for idx, segment in enumerate(segments):
            node = GraphNode(idx)
            valid_morpheme = segment if segment.isupper() else 'root' 
            real_morpheme = valid_morpheme[:-1] if valid_morpheme[-1] =='\n' else valid_morpheme
            appending = Morpheme(
                morpheme = real_morpheme,
                type = "prefix" if not root_processed else "suffix",
                ending = (idx==len(segments)-1),
                count =1,
                probability = 0.0
                )
            if real_morpheme =='root':
                root_processed = True 
            if idx < len(self.graph):
                if appending in self.graph[idx].morphemes:
                    for existing in self.graph[idx].morphemes:
                        if existing ==appending:
                            existing.count += 1 
                else:
                    self.graph[idx].morphemes.add(appending)
            else:
                # if index not present create a new_node
                new_node = GraphNode(idx)
                new_node.morphemes.add(appending)
                self.graph.append(new_node)
    def normalize(self, node) ->None:
        if not node or not node.morphemes:
            return 
        counts = [morpheme.count for morpheme in node.morphemes]
        
        if not counts or max(counts)==0:
            return 
            
        max_count = max(counts) 
        scaled_counts = np.array(counts) / (max_count + 1)
        exp_counts = np.exp(scaled_counts - np.max(scaled_counts))
        total_exp = np.sum(exp_counts)
        
        probabilities = exp_counts / total_exp
        
        for idx, morpheme in enumerate(node.morphemes):
            morpheme.probability = probabilities[idx]
    def get_inflection(self, root)->set:
        inflection = []
        # start with the root and expand to all possible inflections 
        self.get_inflection_helper(root, "", 0, inflection, False)
        return set(inflection)

    def get_inflection_helper(self, root, current: str, index: int, morphemes: list, root_processed: bool):
        if len(current.split("+"))>=8: 
            return 
        if len(morphemes) > 1000:
            return 
        
        if index >= len(self.graph):
            if root_processed:
                morphemes.append(current)
            return 
    
        candidates = sorted(self.graph[index].morphemes, key=lambda morpheme:morpheme.probability)[:5] if index > 2 else self.graph[index].morphemes
        for candidate in candidates:
            new = ""
            temp_root_processed = root_processed
            if candidate.morpheme =="root" and not root_processed:
                new = current + "+" + root 
                temp_root_processed = True 
            else:
                if not root_processed and candidate.type =="prefix":
                    new = current + "+" + candidate.morpheme if candidate.morpheme not in current.split("+") else ""
                elif root_processed and candidate.type == "suffix":
                    new = current + "+" + candidate.morpheme if candidate.morpheme not in current.split("+") else ""

            if candidate.ending and root_processed and len(new)> 0:
                morphemes.append(new)
            else:
                next = new if len(new) > 0 else current
                self.get_inflection_helper(root, next, index+1, morphemes, temp_root_processed)
        
            


#### Testing:  

Testing the segmentation system by building the transition graph and generating inflections for a sample word. This will check if morphemes are processed correctly and if the graph structure makes sense. Printing a few nodes to verify everything looks good.  


In [5]:
segmentation_file = "segmentations.txt"

print("\n Building TransitionGraph from segmentation.txt")
graph = TransitionGraph(segmentation_file)
print(f"Graph built with {len(graph.graph)} nodes.")

test_root = "sura"

print(f"\n  Generating inflections for '{test_root}'")
inflections = graph.get_inflection(test_root)

print(f"Generated {len(inflections)} Inflections for '{test_root}'. Showing first 10:")
print(list(inflections)[:10])  

print("\n Checking individual graph nodes and morphemes")
for i, node in enumerate(graph.graph[:5]):  
    print(f"\nNode {i}: Showing 3 morphemes")
    for morpheme in list(node.morphemes)[:3]: 
        print(f"- {str(morpheme)}")



 Building TransitionGraph from segmentation.txt
Graph built with 12 nodes.

  Generating inflections for 'sura'
Generated 135 Inflections for 'sura'. Showing first 10:
['+V+CL16+HAB.PAST+OBJ.CL11+sura+RECIP+LOC', '+V+CL16+HAB.PAST+OBJ.CL15+sura+BEN+IMP', '+V+CL16+HAB.PAST+OBJ.1PL+sura+RECIP+LOC', '+V+CL16+HAB.PAST+OBJ.CL10+OBJ.1PL+sura+PERF', '+V+CL16+HAB.PAST+OBJ.CL10+OBJ.CL15+sura+IMP', '+V+CL16+HAB.PAST+OBJ.1PL+sura+root+PERF', '+V+CL16+HAB.PAST+OBJ.CL15+sura+root+IMP', '+V+CL16+HAB.PAST+OBJ.CL10+OBJ.CL11+sura+IMP', '+V+CL16+HAB.PAST+OBJ.CL11+sura+PERF', '+V+CL16+HAB.PAST+OBJ.CL15+sura+BEN+PERF']

 Checking individual graph nodes and morphemes

Node 0: Showing 3 morphemes
- morpheme: V, Count: 45957, Probability: 0.5758, ending: False
- morpheme: ADJ, Count: 2, Probability: 0.2119, ending: False
- morpheme: N, Count: 103, Probability: 0.2123, ending: False

Node 1: Showing 3 morphemes
- morpheme: CL16, Count: 634, Probability: 0.0297, ending: False
- morpheme: CL13, Count: 773, Pro